# (저와 같은) 초보 중에서도 초보를 위한 베이스라인

### 저와 같은 초보 중에서도 초보들이 저처럼 시간낭비?를 하지 않기를 바라는 마음으로 이렇게 코드공유를 해봅니다.

####! 이미 한번이라도 제출해본 적이 있으신 분은 안보고 넘어가셔도 좋습니다 !



이 코드 공유는 아래의 어려움을 겪으신 분들을 위한 베이스라인? 코드입니다.
- 데이콘이라는걸 처음 알게 된 분들이 **경험삼아 한번이라도 제출하고 싶은 경우**
- 베이스라인대로 코드를 돌려봤는데 데이터프레임 칼럼명(헤더)가 깨져서 당황하신 분들
- 제공되는 기본 베이스라인코드를 그대로 돌려봤는데 본인이 코드 실행해서 만들어진 **submission.csv와 sample_submission.csv가 서로 다르게 생겨서** 당황하신 분들
- sklearn에서 **GridSearchCV**를 써본적이 없는 분들
- **SMAPE**를 어떻게 loss함수로 지정해야하는지 몰라 포기하셨던 분들



거의 대다수의 분들은 위의 사항에 해당되지 않음을 알고있습니다...만 그래도 저같이 판다스 람다함수 같은 기본적인 것도 못 외워서 검색하고 어디를 실수했는지 못찾아서 거의 하루를 매달려야 하는 분들도 있기는 있을 것이라고 생각하여 올려봅니다


(주의) 기존 베이스라인코드를 많이 베껴왔습니다.

# 데이터 전처리

In [ ]:
#Library Imports
import numpy as np  # 넘파이
import pandas as pd  # 판다스
# import math
import os  # 디렉토리 변경
from sklearn.model_selection import GridSearchCV  # 파라미터 설정 고민을 줄여주는 고마운 친구
from sklearn.metrics import make_scorer  # loss function 커스터마이징

from lightgbm import LGBMRegressor  # 전 LGBMRegressor 만을 사용해서 돌려보았습니다.
from sklearn.model_selection import train_test_split  # train, valid set 제작



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 로드 (인코딩은 euc-kr)

train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/전력수요예측/전처리 다르게 해보기/train_revise.csv', encoding='cp949') #한글 문자열 코딩 시 사용
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/전력수요예측/전처리 다르게 해보기/test3.csv', encoding='cp949') #pd.read_csv 폴더 내의 csv를 읽어오는 것이다. Colab 말고 Jupyter notebook으로 할 때 주의!
submission=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/전력수요예측/전처리 다르게 해보기/sample_submission.csv', encoding='cp949')

In [ ]:
train.head(7)

,Unnamed: 0,num,date_time,power,temp,wind,hum,prec,sun,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,6,23,8528.627077,8540.373176,118.793252,0,0.000000,1.000000e+00,57.5376,-8.4
1,1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0,6,23,8513.723077,8517.174776,137.989738,0,0.258819,9.659258e-01,57.0389,-16.7
2,2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0,6,23,8496.625846,8509.055718,122.381197,0,0.500000,8.660254e-01,56.6789,-25.2
3,3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0,6,23,8480.076923,8493.313129,122.054777,0,0.707107,7.071068e-01,55.9589,-34.1
4,4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0,6,23,8472.051692,8479.522165,124.472447,0,0.866025,5.000000e-01,56.4576,-43.1
5,5,1,2020-06-01 05,8010.576,16.9,3.4,93.0,0.0,0.0,0,6,23,8444.088000,8454.715200,120.468353,0,0.965926,2.588190e-01,56.9761,-52.2
6,6,1,2020-06-01 06,7978.176,16.7,3.4,90.0,0.1,0.0,0,6,23,8440.150154,8446.512282,122.079008,0,1.000000,6.123234e-17,54.5800,-61.5


In [ ]:
test.head(7)  # 전력사용량 없음

,num,date_time,power,temp,wind,hum,prec,sun,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,1,2020-08-25 00,NaN,27.8,1.5,74.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,NaN,27.3,1.1,78.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,2020-08-25 05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,2020-08-25 06,NaN,26.1,1.3,83.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 기온, 풍속, 습도 등, 기타 결측치를 적당히 1/3, 2/3 수치로 보간해줍니다.
test = test.interpolate(method='values')  
test.head(3)

# 모델링

In [ ]:
# 학습용set 생성
train.drop('date_time', axis=1, inplace=True)  # 학습에 불필요한 날짜 제거
train_x=train.drop('power', axis=1)  # 문제
train_y=train[['power']]  # 정답

X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=156)

In [ ]:
X_train.head(1)

,Unnamed: 0,num,temp,wind,hum,prec,sun,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
91868,91868,46,21.0,5.2,80.0,0.0,0.0,2,6,23,1751.724,1621.219765,227.480473,0,-0.866025,0.5,56.82,-13.7


In [ ]:
y_train.head(1)

,power
91868,1609.632


In [ ]:
# loss function : SMAPE 정의
# from sklearn.metrics import mean_absolute_error
def smape(true, pred):
    true = np.array(true)  # np.array로 바꿔야 에러 없음
    pred = np.array(pred)
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred)))  # *2 , *100은 상수이므로 생략
SMAPE = make_scorer(smape, greater_is_better=False)  # smape 값이 작아져야하므로 False

In [ ]:
# 파라미터 설정, 모델생성 함수
def get_best_params(model, params):
    grid_model = GridSearchCV(
        model,
        param_grid = params,  # 파라미터
        cv=5,  # Kfold : 5
        scoring= SMAPE)  #loss function

    grid_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=100)
    scr = grid_model.best_score_
    print(f'{model.__class__.__name__} 최적 score 값 {scr}')
    return grid_model.best_estimator_

In [ ]:

params = {}
params['boosting_type'] = ['gbdt']
params['objective'] = ['regression']
params['n_estimators'] = [100]
params['learning_rate'] = [0.1, 0.01]  
params['subsample'] = [1]

In [ ]:
# 모델정의
model=LGBMRegressor(params)

In [ ]:
# 학습진행
best_lgbm = get_best_params(model, params)
best_lgbm  # learning_rate 0.1 or 0.01 중, 0.01이 더 좋았다고 하는군요. 

[100]	valid_0's l2: 28930.9	valid_1's l2: 34593.7
[100]	valid_0's l2: 29238	valid_1's l2: 35586.7
[100]	valid_0's l2: 29725.2	valid_1's l2: 35629
[100]	valid_0's l2: 28953.2	valid_1's l2: 36042.8
[100]	valid_0's l2: 28794.9	valid_1's l2: 34567.1
[100]	valid_0's l2: 624826	valid_1's l2: 660345
[100]	valid_0's l2: 624797	valid_1's l2: 660219
[100]	valid_0's l2: 623598	valid_1's l2: 658008
[100]	valid_0's l2: 625307	valid_1's l2: 661190
[100]	valid_0's l2: 625081	valid_1's l2: 660723
[100]	training's l2: 624277	valid_1's l2: 659338
LGBMRegressor 최적 score 값 -0.3115989716350526


LGBMRegressor(learning_rate=0.01, objective='regression', subsample=1)

In [ ]:
# 기타 지표로 에러 측정
from sklearn.metrics import mean_squared_error, r2_score
y_pred = best_lgbm.predict(X_train)

mse_score = mean_squared_error(y_train, y_pred)
r2_score = r2_score(y_train, y_pred)
print('MSE:', mse_score)
print('R2 :', r2_score)

MSE: 624276.9821285354
R2 : 0.8510478951110365


####  MSE: 1137593.8469961581 =>  624276.98 (낮을 수록 좋은거임)
####  R2 : 0.7285708061170786 => 0.85104 (높을수록 좋은거임)


In [ ]:
# 모델저장, 로드
import joblib
joblib.dump(best_lgbm, 'best_lgbm.pkl')
load_lgbm = joblib.load('best_lgbm.pkl')

# 제출용 데이터

In [ ]:
# 모델에 넣기 위해 날짜칼럼 제거
test_x = test.drop('date_time', axis=1)

In [ ]:
# 모델 예측
submission_y = best_lgbm.predict(test_x)

In [ ]:
# submission.csv 생성
test['answer'] = submission_y
test['num_date_time'] = test.apply(lambda x: str(x['num']) +' '+ x['date_time'], axis=1)
submission = test[['num_date_time', 'answer']]
submission.to_csv('submission.csv', index=False)
submission

,num_date_time,answer
0,1 2020-08-25 00,1154.188837
1,1 2020-08-25 01,1154.188837
2,1 2020-08-25 02,1154.188837
3,1 2020-08-25 03,1154.188837
4,1 2020-08-25 04,1154.188837
...,...,...
10075,60 2020-08-31 19,1154.188837
10076,60 2020-08-31 20,1154.188837
10077,60 2020-08-31 21,1154.188837
10078,60 2020-08-31 22,1154.188837
